# 2.1 Metadata

Author: Sandra Godinho Silva \
Creation date: 07/09/2020 \
Version: 0.1

In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 10)

## Import dataset

In [5]:
dataset = pd.read_csv("../1_Dataset_creation/Dataset.csv")
dataset.head(2)

,Genome_ID,Bin_Id,Marker lineage,Completeness,Contamination,Strain_heterogeneity,Quality_score,classification,Domain,Phyla,...,scaf_L90,ctg_N90,ctg_L90,scaf_max,ctg_max,scaf_n_gt50K,scaf_pct_gt50K,gc_avg,gc_std,Classification_quality
0,GCA_000016645.1,GCA_000016645.1_ASM1664v1_genomic,f__Flavobacteriaceae (UID2817),99.65,0.14,0.0,98.95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,...,6096872,1,6096872,6096872,6096872,1,100.0,0.34113,0.0,Medium
1,GCA_000023285.1,GCA_000023285.1_ASM2328v1_genomic,p__Bacteroidetes (UID2605),100.00,0.00,0.0,100.00,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,...,2612925,1,2612925,2612925,2612925,1,100.0,0.39588,0.0,Medium


In [4]:
genomes = dataset["Genome_ID"]
genomes.to_csv("assembly_ids.csv", index=False,  header=False)

In [5]:
len(genomes)

2680

## 2.1 Import metadata coupled with GenBank

***On the command line:*** \
wget ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_genbank.txt

In [6]:
genbank_metadata = pd.read_csv("assembly_summary_genbank.txt", sep="\t", skiprows=1)

In [7]:
genbank_metadata.head(2)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material
0,GCA_000001215.4,PRJNA13812,SAMN02803731,NaN,reference genome,7227,7227,Drosophila melanogaster,NaN,NaN,latest,Chromosome,Major,Full,2014/08/01,Release 6 plus ISO1 MT,The FlyBase Consortium/Berkeley Drosophila Gen...,GCF_000001215.4,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN
1,GCA_000001405.28,PRJNA31257,NaN,NaN,reference genome,9606,9606,Homo sapiens,NaN,NaN,latest,Chromosome,Patch,Full,2019/02/28,GRCh38.p13,Genome Reference Consortium,GCF_000001405.39,different,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN


In [8]:
genbank_filtered = pd.merge(genomes, genbank_metadata, how="left", left_on="Genome_ID", right_on="# assembly_accession", indicator="merge")
genbank_filtered["merge"].value_counts()

both          2679
left_only        1
right_only       0
Name: merge, dtype: int64

In [9]:
genbank_filtered.drop(columns=["merge"], inplace=True)
genbank_filtered.head(2)

,Genome_ID,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material
0,GCA_000016645.1,GCA_000016645.1,PRJNA16082,SAMN02598357,NaN,representative genome,376686.0,986.0,Flavobacterium johnsoniae UW101,strain=UW101; ATCC 17061,NaN,latest,Complete Genome,Major,Full,2007/05/01,ASM1664v1,US DOE Joint Genome Institute,GCF_000016645.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material
1,GCA_000023285.1,GCA_000023285.1,PRJNA29403,SAMN00001911,NaN,representative genome,521097.0,1018.0,Capnocytophaga ochracea DSM 7271,strain=DSM 7271,NaN,latest,Complete Genome,Major,Full,2009/08/26,ASM2328v1,US DOE Joint Genome Institute (JGI-PGF),GCF_000023285.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material


In [10]:
#REMOVE empty columns
null_cols = genbank_filtered.isnull().all()
genbank_filtered = genbank_filtered[genbank_filtered.columns[~null_cols]]

In [11]:
#genbank_filtered.info()

## 2.2 Import Metadata from PATRIC

Download Genome features for the taxon Flavobacteriia on 7 September 2020

In [12]:
patric = pd.read_csv("PATRIC_genome_Flavobacteria.csv")

In [13]:
l = patric.columns.to_list()
print(l)

['Genome ID', 'Genome Name', 'Organism Name', 'NCBI Taxon ID', 'Genome Status', 'Strain', 'Serovar', 'Biovar', 'Pathovar', 'MLST', 'Other Typing', 'Culture Collection', 'Type Strain', 'Completion Date', 'Publication', 'BioProject Accession', 'BioSample Accession', 'Assembly Accession', 'SRA Accession', 'GenBank Accessions', 'RefSeq Accessions', 'Sequencing Centers', 'Sequencing Status', 'Sequencing Platform', 'Sequencing Depth', 'Assembly Method', 'Chromosomes', 'Plasmids', 'Contigs', 'Genome Length', 'GC Content', 'PATRIC CDS', 'RefSeq CDS', 'Isolation Site', 'Isolation Source', 'Isolation Comments', 'Collection Date', 'Isolation Country', 'Geographic Location', 'Latitude', 'Longitude', 'Altitude', 'Depth', 'Other Environmental', 'Host Name', 'Host Gender', 'Host Age', 'Host Health', 'Body Sample Site', 'Body Sample Subsite', 'Other Clinical', 'AntiMicrobial Resistance', 'AntiMicrobial Resistance Evidence', 'Gram Stain', 'Cell Shape', 'Motility', 'Sporulation', 'Temperature Range', 'O

In [14]:
#REMOVE uninteresting columns
patric = patric.drop(columns=['Genome ID','PATRIC CDS','Coarse Consistency', 'Fine Consistency', 'Checkm Completeness', 'Checkm Contamination','Genome Quality','Date Inserted', 'Date Modified'])

In [15]:
#REMOVE empty columns
null_cols = patric.isnull().all()
patric = patric[patric.columns[~null_cols]]

In [16]:
patric.head(2)

,Genome Name,Organism Name,NCBI Taxon ID,Genome Status,Strain,Serovar,Pathovar,MLST,Other Typing,Culture Collection,Type Strain,Completion Date,Publication,BioProject Accession,BioSample Accession,Assembly Accession,SRA Accession,GenBank Accessions,RefSeq Accessions,Sequencing Centers,Sequencing Status,Sequencing Platform,Sequencing Depth,Assembly Method,Chromosomes,Plasmids,Contigs,Genome Length,GC Content,RefSeq CDS,Isolation Source,Isolation Comments,Collection Date,Isolation Country,Geographic Location,Latitude,Longitude,Altitude,Depth,Other Environmental,Host Name,Host Gender,Host Age,Host Health,Body Sample Site,Other Clinical,Gram Stain,Cell Shape,Motility,Sporulation,Temperature Range,Optimal Temperature,Salinity,Oxygen Requirement,Habitat,Disease,Comments,Additional Metadata,Genome Quality Flags
0,Muricauda taeanensis strain JCM 17757,NaN,1005926,WGS,JCM 17757,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-20T00:00:00Z,NaN,PRJNA437398,SAMN09939642,GCA_003584105.1,NaN,QXFO01000000,NaN,"Second Institute of Oceanography, State Oceani...",NaN,Illumina HiSeq X-Ten,308x,ABySS v. 1.9.0,NaN,NaN,28,4101660.0,45.60536,3644.0,tidal mudflat,NaN,NaN,South Korea,South Korea: Taean coast of the Yellow Sea,NaN,NaN,NaN,5m,env_biome:marine biome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This project was used for calculating ANI valu...,NaN,NaN
1,Myroides injenensis M09-0166,Myroides injenensis M09-0166,1008457,WGS,NaN,NaN,NaN,NaN,NaN,,NaN,2012-02-04T00:00:00Z,22535932,PRJDA171989,SAMD00036610,GCA_000246945.2,NaN,BAEX00000000,-,Korea Research Institute of Bioscience and Bio...,WGS,454,36x,NaN,NaN,NaN,226,3458160.0,31.35000,NaN,urine of a patient with fever,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Human, Homo sapiens",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Myroides strains are not normal human micr...,NaN,NaN


In [17]:
patric_filtered = pd.merge(genomes, patric, how="left", left_on="Genome_ID", right_on="Assembly Accession", indicator="merge")
patric_filtered["merge"].value_counts()

both          2081
left_only      605
right_only       0
Name: merge, dtype: int64

In [18]:
patric_filtered["Additional Metadata"] = patric_filtered["Additional Metadata"].str.replace("�","")
patric_filtered.drop(columns=["merge"], inplace=True)
patric_filtered.head(2)

,Genome_ID,Genome Name,Organism Name,NCBI Taxon ID,Genome Status,Strain,Serovar,Pathovar,MLST,Other Typing,Culture Collection,Type Strain,Completion Date,Publication,BioProject Accession,BioSample Accession,Assembly Accession,SRA Accession,GenBank Accessions,RefSeq Accessions,Sequencing Centers,Sequencing Status,Sequencing Platform,Sequencing Depth,Assembly Method,Chromosomes,Plasmids,Contigs,Genome Length,GC Content,RefSeq CDS,Isolation Source,Isolation Comments,Collection Date,Isolation Country,Geographic Location,Latitude,Longitude,Altitude,Depth,Other Environmental,Host Name,Host Gender,Host Age,Host Health,Body Sample Site,Other Clinical,Gram Stain,Cell Shape,Motility,Sporulation,Temperature Range,Optimal Temperature,Salinity,Oxygen Requirement,Habitat,Disease,Comments,Additional Metadata,Genome Quality Flags
0,GCA_000016645.1,Flavobacterium johnsoniae UW101,Flavobacterium johnsoniae UW101,376686.0,Complete,ATCC 17061,NaN,NaN,NaN,NaN,ATCC 17061,Yes,2007-05-01T00:00:00Z,NaN,PRJNA16082,SAMN02598357,GCA_000016645.1,NaN,CP000685,NC_009441,DOE Joint Genome Institute,complete,NaN,NaN,NaN,1.0,NaN,1.0,6096872.0,34.1,5017.0,soil,isolated from soil in England and is the type ...,NaN,United Kingdom,United Kingdom: England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,Rod,Yes,No,Mesophilic,20-30,Non-halophilic,Aerobic,Multiple,NaN,Flavobacterium johnsoniae ATCC 17061. Flavobac...,NaN,NaN
1,GCA_000023285.1,Capnocytophaga ochracea DSM 7271,Capnocytophaga ochracea DSM 7271,521097.0,Complete,DSM 7271,NaN,NaN,NaN,NaN,DSM 7271,Yes,2009-04-30T00:00:00Z,21304645,PRJNA29403,SAMN00001911,GCA_000023285.1,"SRR013476,SRR013477",CP001632,NC_013162,US DOE Joint Genome Institute (JGI-PGF)|DOE Jo...,complete,NaN,NaN,NaN,1.0,NaN,1.0,2612925.0,39.6,2171.0,the human oral cavity,isolated from the human oral cavity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Human, Homo sapiens",NaN,NaN,NaN,NaN,NaN,-,Rod,Yes,No,Mesophilic,35-37,NaN,Facultative,Host-associated,NaN,Capnocytophaga ochracea DSM 7271. Capnocytoph...,NaN,NaN


In [19]:
#REMOVE empty columns
null_cols = patric_filtered.isnull().all()
patric_filtered = patric_filtered[patric_filtered.columns[~null_cols]]

### 2.2.1 Split Patric column: Additional Metadata

In [21]:
assembly_accession = patric_filtered["Genome_ID"].to_list()

In [22]:
splitted = patric_filtered["Additional Metadata"].str.split(";", expand=True)
splitted = splitted.rename(index=patric_filtered["Genome_ID"])
splitted = splitted.where(pd.notnull(splitted), None)

In [23]:
col_list = []
for i, cell in splitted.iteritems():
    for x in cell:
        if x is None:   #while y == nan:
            continue
        else:
            key = x.split(":")[0]
            value = x.split(":")[1:]
            if key not in col_list:
                col_list.append(key)
print(col_list)

['collected_by', 'sample_type', 'biomaterial_provider', 'identified_by', 'passage_history', 'isolate', 'specimen_voucher', '', 'lab_host', 'mating_type']


In [24]:
patric_addit_metadata = pd.DataFrame(columns=col_list, index=patric_filtered["Genome_ID"])

In [25]:
for i, cell in splitted.iterrows():
    for x in cell:
        cell = x
        if x is None:
            continue
        else:
            if ":" in x:
                value = x.split(":")[1]
                key = x.split(":")[0]
                if key in col_list:
                    patric_addit_metadata.loc[i,key] = str(value)

In [26]:
patric_addit_metadata = patric_addit_metadata.reset_index()
patric_addit_metadata = patric_addit_metadata.drop_duplicates(subset=['Genome_ID'])
patric_addit_metadata.tail(2)

,Genome_ID,collected_by,sample_type,biomaterial_provider,identified_by,passage_history,isolate,specimen_voucher,,lab_host,mating_type
2684,GCA_903819445.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2685,GCA_903969135.1,Yvan Rahb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Biosamples

BioSamples stores and supplies descriptions and metadata about biological samples used in research and development by academia and industry. Samples are either 'reference' samples (e.g. from 1000 Genomes, HipSci, FAANG) or have been used in an assay database such as the European Nucleotide Archive (ENA) or ArrayExpress.

In [27]:
biosamples = pd.read_csv("Biosamples_db.csv", low_memory=False)
l = biosamples.columns.to_list()
#print(l)

In [28]:
biosamples.head(2)

,biosample_title,contact_email,first_name,growth phase,last_name,name,organism,publication_date,sample_accession,source_name,strain,submission_model,submission_package,submission_package_name,submitter,taxonomy_id,treatment,collection_date,culture_collection,geo_loc_name,isolation_source,sample_name,sample_type,type-material,comment_paragraph,depth,derived-from,env_broad_scale,env_local_scale,env_medium,environmental-sample,isolate,lat_lon,metagenome-source,metagenomic,sample-type,elev,environmetal-sample,collected_by,host,host_tissue_sampled,temp,env_biome,env_feature,env_material,isol_growth_condt,locus_tag_prefix,num_replicons,project_name,ref_biomaterial,16S recovered,16S recovery software,analysis project type,assembly quality,assembly software,bin parameters,binning software,completeness score,completeness software,contamination score,contig L50,decontamination software,note,number of contigs,reassembly post binning,relative coverage Anaerobic (SRR10097246),relative coverage Anoxic (SRR10375098),relative coverage Infiltration (SRR10097247),relative coverage Low pH (SRR10097245),relative coverage Oxic (SRR10375099),relative coverage Reference (SRR10375097),taxa id,total assembly size,nat_host,assembly_method_and_version,completeness_estimated,contamination_estimated,genome_coverage,mapping_method_and_version,quality_assessment_method_and_version,ArrayExpress-SPECIES,ENA-FIRST-PUBLIC,ENA-LAST-UPDATE,SUBJECT_ID,contact_lab,identified_by,isolate-name-alias,environmental_sample,metagenome_source,biomaterial_provider,culture media,replicate,biome,feature,material,alternate_ID,culture-collection,altitude,Genus,ProjectAccession,PublicAccession,Species,attribute_package,isolate_name_alias,host_disease,rel_to_oxygen,Alias,ENA checklist,INSDC center alias,INSDC center name,INSDC first public,INSDC last update,INSDC status,SRA accession,Sample Name,Title,serotype,subsrc_note,annotation_method,assembly_method,biosamplemodel,geographic location (country and/or atlantic ocean),geographic location (region and locality),mapping_method,quality_assessment_method,refinement_method,Cell Shape,Funding Program,GOLD Stamp ID,Gene Calling Method,Gram Staining,Isolation Comments,Isolation Site,Motility,Temperature Optimum,Temperature Range,Type Strain,alt_elev,biotic_relationship,country,environment,investigation_type,isolation-source,trophic_level,pH,Body Sample Site,Body Sample SubSite,Diseases,Host Name,Host Health,Sporulation,experimental_factor,breeding_method,lat-lon,ENA-CHECKLIST,External Id,Submitter Id,subsrc-note,collection date,environment (biome),environment (feature),environment (material),geographic location (country and/or sea),geographic location (latitude),geographic location (longitude),host common name,host-associated environmental package,investigation type,project name,sequencing method,STRAIN,co_assembly_bin,derived from,description,Sequence processing,completeness,contamination,host_age,host_sex,oxy_stat_samp,samp_store_dur,samp_store_temp,FDA CVM ID,bio_material,common name,host scientific name,samp_mat_process,estimated_size,ArrayExpress-STRAIN_OR_LINE,Tax ID,Collaborator ID,host_description,host_disease_outcome,host_disease_stage,host_health_state,identification method,strain_name_alias,WGA amplification approach,metagenomic source,sample derived from,single cell or viral particle lysis approach,sorting technology,taxonomic identity marker,passage_history,samp_size,timepoint,Description,subsource_note,lab_host,collection_room,relative_week,assembly_tool,bin_id,Broker name,number_of_identified_antimicrobial_resistance_genes,MAG_number,derived_from,alkalinity (carbonate/bicarbonate),salinity,original-organism-name,Phenotypes,env_package,label,specific_host,binning parameters,plant-associated environmental package,culture collection,repository,risk group,type status,Assembly Method,Genome Coverage,Sequencing Technology,nat-host,collection-date,Historical Monobactam Produced,Unique Monobactam Clusters Identified,extrachrom_e

In [29]:
biosamples_filtered = pd.merge(patric_filtered, biosamples, how="left", left_on="BioSample Accession", right_on="sample_accession", indicator="merge")
biosamples_filtered["merge"].value_counts()

both          1973
left_only      713
right_only       0
Name: merge, dtype: int64

In [30]:
biosamples_filtered.drop(columns=["merge"], inplace=True)
biosamples_filtered.head(2)

,Genome_ID,Genome Name,Organism Name,NCBI Taxon ID,Genome Status,Strain_x,Serovar,Pathovar,MLST,Other Typing,Culture Collection,Type Strain_x,Completion Date,Publication,BioProject Accession,BioSample Accession,Assembly Accession,SRA Accession,GenBank Accessions,RefSeq Accessions,Sequencing Centers,Sequencing Status,Sequencing Platform,Sequencing Depth,Assembly Method_x,Chromosomes,Plasmids,Contigs,Genome Length,GC Content,RefSeq CDS,Isolation Source,Isolation Comments_x,Collection Date,Isolation Country,Geographic Location,Latitude,Longitude,Altitude,Depth_x,Other Environmental,Host Name_x,Host Gender,Host Age,Host Health_x,Body Sample Site_x,Other Clinical,Gram Stain,Cell Shape_x,Motility_x,Sporulation_x,Temperature Range_x,Optimal Temperature,Salinity_x,Oxygen Requirement,Habitat,Disease,Comments,Additional Metadata,Genome Quality Flags,biosample_title,contact_email,first_name,growth phase,last_name,name,organism,publication_date,sample_accession,source_name,strain,submission_model,submission_package,submission_package_name,submitter,taxonomy_id,treatment,collection_date,culture_collection,geo_loc_name,isolation_source,sample_name,sample_type,type-material,comment_paragraph,depth,derived-from,env_broad_scale,env_local_scale,env_medium,environmental-sample,isolate,lat_lon,metagenome-source,metagenomic,sample-type,elev,environmetal-sample,collected_by,host,host_tissue_sampled,temp,env_biome,env_feature,env_material,isol_growth_condt,locus_tag_prefix,num_replicons,project_name,ref_biomaterial,16S recovered,16S recovery software,analysis project type,assembly quality,assembly software,bin parameters,binning software,completeness score,completeness software,contamination score,contig L50,decontamination software,note,number of contigs,reassembly post binning,relative coverage Anaerobic (SRR10097246),relative coverage Anoxic (SRR10375098),relative coverage Infiltration (SRR10097247),relative coverage Low pH (SRR10097245),relative coverage Oxic (SRR10375099),relative coverage Reference (SRR10375097),taxa id,total assembly size,nat_host,assembly_method_and_version,completeness_estimated,contamination_estimated,genome_coverage,mapping_method_and_version,quality_assessment_method_and_version,ArrayExpress-SPECIES,ENA-FIRST-PUBLIC,ENA-LAST-UPDATE,SUBJECT_ID,contact_lab,identified_by,isolate-name-alias,environmental_sample,metagenome_source,biomaterial_provider,culture media,replicate,biome,feature,material,alternate_ID,culture-collection,altitude,Genus,ProjectAccession,PublicAccession,Species,attribute_package,isolate_name_alias,host_disease,rel_to_oxygen,Alias,ENA checklist,INSDC center alias,INSDC center name,INSDC first public,INSDC last update,INSDC status,SRA accession,Sample Name,Title,serotype,subsrc_note,annotation_method,assembly_method,biosamplemodel,geographic location (country and/or atlantic ocean),geographic location (region and locality),mapping_method,quality_assessment_method,refinement_method,Cell Shape_y,Funding Program,GOLD Stamp ID,Gene Calling Method,Gram Staining,Isolation Comments_y,Isolation Site,Motility_y,Temperature Optimum,Temperature Range_y,Type Strain_y,alt_elev,biotic_relationship,country,environment,investigation_type,isolation-source,trophic_level,pH,Body Sample Site_y,Body Sample SubSite,Diseases,Host Name_y,Host Health_y,Sporulation_y,experimental_factor,breeding_method,lat-lon,ENA-CHECKLIST,External Id,Submitter Id,subsrc-note,collection date,environment (biome),environment (feature),environment (material),geographic location (country and/or sea),geographic location (latitude),geographic location (longitude),host common name,host-associated environmental package,investigation type,project name,sequencing method,STRAIN,co_assembly_bin,derived from,description,Sequence processing,completeness,contamination,host_age,host_sex,oxy_stat_samp,samp_store_dur,samp_store_temp,FDA CVM ID,bio_material,common name,host scientific name,samp_mat_process,estimated_size,ArrayExpress-STRAIN_OR_LINE,Tax ID,Collaborat

In [31]:
#REMOVE empty columns
null_cols = biosamples_filtered.isnull().all()
biosamples_filtered = biosamples_filtered[biosamples_filtered.columns[~null_cols]]

In [32]:
biosamples_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2686 entries, 0 to 2685
Columns: 377 entries, Genome_ID to locus_tag prefix
dtypes: float64(34), object(343)
memory usage: 7.7+ MB


# 2.4 SRA

**SRA data manipulation was done in jupyter notebook:** 2.1.1_SRA_attribute

In [33]:
sra = pd.read_csv("SRA_metadata.csv")
sra.head(2)

,assembly_accession,strain,finishing strategy (depth of coverage),biotic_relationship,geo_loc_name,host,collected_by,host_disease,isolation_source,sample_name,isolation-source,sample_type,FDA CVM ID,culture_collection,assembly_method,collection_date,env_feature,synonym,Strain,SUBJECT_ID,Alias,type-material,misc_param,collection date,Isolation Site,Isolation source,country,note,GOLD Stamp ID,investigation_type,culture-collection,lat_lon,alt_elev,locus_tag_prefix,nat-host,assembly_method_version,env_material,model,sample_description,Sample Description,ArrayExpress-STRAIN_OR_LINE,ENA checklist,project_type,estimated_size,Temperature Optimum,isol_growth_condt,Motility,feature,project_name,assembly,num_replicons,isolate,depth,env_biome,altitude,ref_biomaterial,package,geographic location,ArrayExpress-StrainOrLine,ArrayExpress-Phenotype,ArrayExpress-SPECIES,INSDC center alias,geo-loc-name,environment,host_sex,elev,PublicAccession,completeness_estimated,geographic location (country and/or sea),latitude and longitude,specific_host,ArrayExpress-Species,ENA-FIRST-PUBLIC,INSDC center name,specific host,Body Sample Site,pH,biome,biomaterial_provider,isolate_name_alias,contamination_estimated,host scientific name,anonymized_name,ENA-LAST-UPDATE,INSDC first public,Cell Shape,BioSampleModel,environmental_sample,material,identified_by,host health state,INSDC last update,Diseases,strain_name_alias,genome_coverage,INSDC status,host taxid,isolation source,Gram Staining,Temperature Range,Sporulation,sequencing_meth,Historical Monobactam Produced,identification method,supplier_name,SRA accession,host_disease_outcome,Unique Monobactam Clusters Identified,ProjectAccession,Sample Name,Type Strain,Phenotypes,Gene Calling Method,temp,Title,investigation type,Host,rel_to_oxygen,host_age,Species,mapping_method,serovar,sop,Isolation Comments,finishing_strategy,Genus,mapping_method_version,environment (biome),attribute_package,metagenome_source,environment (feature),trophic_level,metagenomic,environment (material),Funding Program,quality_assessment_method,quality_assessment_method_version,geographic location (latitude),source_mat_id,host_tissue_sampled,geographic location (longitude),Host Name,subsrc_note,Host Health,value,plant-associated environmental package,project name,sequencing method,Body Sample SubSite
0,GCA_000023285.1,DSM 7271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Capnocytophaga ochracea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GCA_000143785.1,ATCC 35910,"Level 6, Finished33.84x;7",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Chryseobacterium gleum,"HMP body site, Vaginal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATCC:35910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reference Genome,5562000.0,NaN,NaN,NaN,Homo sapiens-associated habitat ENVO:00009003,Chryseobacterium gleum F93,Newbler v. 2.0.0-Eclipse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homo sapiens,NaN,NaN,terrestrial biome ENVO:00000446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIGS.ba,NaN,biological product ENVO:02000043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9606.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://hmpdacc.org/doc/CommonGeneAnnotation_SO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATCC 35910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454-GS-FLX,NaN


----

# 2.5 MERGE

In [34]:
merge1 = pd.merge(genbank_filtered, biosamples_filtered, how="left", left_on="Genome_ID", right_on="Genome_ID", indicator="merge")
merge1["merge"].value_counts()

both          2686
right_only       0
left_only        0
Name: merge, dtype: int64

In [35]:
merge2 = pd.merge(merge1, sra, how="left", left_on="Genome_ID", right_on="assembly_accession", indicator="merge2")
merge2["merge2"].value_counts()

left_only     2149
both           537
right_only       0
Name: merge2, dtype: int64

In [36]:
merge2.head(2)

,Genome_ID,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate_x,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter_x,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,Genome Name,Organism Name,NCBI Taxon ID,Genome Status,Strain_x,Serovar,Pathovar,MLST,Other Typing,Culture Collection,Type Strain_x,Completion Date,Publication,BioProject Accession,BioSample Accession,Assembly Accession,SRA Accession,GenBank Accessions,RefSeq Accessions,Sequencing Centers,Sequencing Status,Sequencing Platform,Sequencing Depth,Assembly Method_x,Chromosomes,Plasmids,Contigs,Genome Length,GC Content,RefSeq CDS,Isolation Source,Isolation Comments_x,Collection Date,Isolation Country,Geographic Location,Latitude,Longitude,Altitude,Depth_x,Other Environmental,Host Name_x,Host Gender,Host Age,Host Health_x,Body Sample Site_x,Other Clinical,Gram Stain,Cell Shape_x,Motility_x,Sporulation_x,Temperature Range_x,Optimal Temperature,Salinity_x,Oxygen Requirement,Habitat,Disease,Comments,Additional Metadata,Genome Quality Flags,biosample_title,contact_email,first_name,last_name,name,organism,publication_date,sample_accession,strain_x,submission_model,submission_package,submission_package_name,submitter_y,taxonomy_id,collection_date_x,culture_collection_x,geo_loc_name_x,isolation_source_x,sample_name_x,sample_type_x,type-material_x,comment_paragraph,depth_x,derived-from,env_broad_scale,env_local_scale,env_medium,environmental-sample,isolate_y,lat_lon_x,metagenome-source,metagenomic_x,sample-type,elev_x,environmetal-sample,collected_by_x,host_x,host_tissue_sampled_x,temp_x,env_biome_x,env_feature_x,env_material_x,isol_growth_condt_x,locus_tag_prefix_x,num_replicons_x,project_name_x,ref_biomaterial_x,16S recovered,16S recovery software,analysis project type,assembly quality,assembly software,bin parameters,binning software,completeness score,completeness software,contamination score,contig L50,decontamination software,note_x,number of contigs,reassembly post binning,relative coverage Anaerobic (SRR10097246),relative coverage Anoxic (SRR10375098),relative coverage Infiltration (SRR10097247),relative coverage Low pH (SRR10097245),relative coverage Oxic (SRR10375099),relative coverage Reference (SRR10375097),taxa id,total assembly size,nat_host,assembly_method_and_version,completeness_estimated_x,contamination_estimated_x,genome_coverage_x,mapping_method_and_version,quality_assessment_method_and_version,ENA-FIRST-PUBLIC_x,ENA-LAST-UPDATE_x,SUBJECT_ID_x,contact_lab,identified_by_x,environmental_sample_x,metagenome_source_x,biomaterial_provider_x,biome_x,feature_x,material_x,culture-collection_x,altitude_x,isolate_name_alias_x,host_disease_x,rel_to_oxygen_x,Alias_x,ENA checklist_x,INSDC center alias_x,INSDC center name_x,INSDC first public_x,INSDC last update_x,INSDC status_x,SRA accession_x,Sample Name_x,Title_x,serotype,subsrc_note_x,annotation_method,assembly_method_x,biosamplemodel,geographic location (country and/or atlantic ocean),geographic location (region and locality),mapping_method_x,quality_assessment_method_x,refinement_method,Cell Shape_y,Funding Program_x,GOLD Stamp ID_x,Gene Calling Method_x,Gram Staining_x,Isolation Comments_y,Isolation Site_x,Motility_y,Temperature Optimum_x,Temperature Range_y,Type Strain_y,alt_elev_x,biotic_relationship_x,country_x,environment_x,investigation_type_x,isolation-source_x,trophic_level_x,pH_x,Body Sample Site_y,Body Sample SubSite_x,Diseases_x,Host Name_y,Host Health_y,Sporulation_y,experimental_factor,lat-lon,ENA-CHECKLIST,External Id,Submitter Id,subsrc-note,collection date_x,environment (biome)_x,environment (feature)_x,environment (material)_x,geographic location (country and/or sea)_x,geographic location (latitude)_x,geographic location (longitude)_x,host common name,host-associated environmental package,investigation type_x,project name_x,sequencing method_x,STRAIN,

In [37]:
merge2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2686 entries, 0 to 2685
Columns: 550 entries, Genome_ID to merge2
dtypes: category(2), float64(49), object(499)
memory usage: 11.3+ MB


In [38]:
merge3 = pd.merge(merge2, patric_addit_metadata, how="left", left_on="Genome_ID", right_on="Genome_ID", indicator="merge3", suffixes="_z")
merge3["merge3"].value_counts()

both          2686
right_only       0
left_only        0
Name: merge3, dtype: int64

In [39]:
merge = merge3.drop(columns=["merge", "merge2", "merge3", "Additional Metadata"])

In [40]:
to_replace=[r"^missing$", r"^Missing$",  r"^'Missing'", 
            r"^Unclassified$", r"^unclassified$",
            r"^NA$", r"^na$", r'^None$',
            r"^not determined$", r"^Not determined$",
            r"^Not collected$", r"^not collected$", r'^Not Collected$'
            r'^Not recorded$', r'^not recorded$',r'^Not Recorded$', 
            r"^unspecified$", r"^Unspecified$",
            r"^Not Applicable$",r"^Not applicable$", r'^not applicable$',
            r"^not available: to be reported later$", r"^not available$",
            r"^Unknown", r"^unknown", 
            r'^"$', r"^'$", r'^_$',r'^-$'
            ]

In [41]:
merge = merge.replace({w: np.nan for w in to_replace}, regex=True)

In [42]:
merge.head(2)

,Genome_ID,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate_x,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter_x,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,Genome Name,Organism Name,NCBI Taxon ID,Genome Status,Strain_x,Serovar,Pathovar,MLST,Other Typing,Culture Collection,Type Strain_x,Completion Date,Publication,BioProject Accession,BioSample Accession,Assembly Accession,SRA Accession,GenBank Accessions,RefSeq Accessions,Sequencing Centers,Sequencing Status,Sequencing Platform,Sequencing Depth,Assembly Method_x,Chromosomes,Plasmids,Contigs,Genome Length,GC Content,RefSeq CDS,Isolation Source,Isolation Comments_x,Collection Date,Isolation Country,Geographic Location,Latitude,Longitude,Altitude,Depth_x,Other Environmental,Host Name_x,Host Gender,Host Age,Host Health_x,Body Sample Site_x,Other Clinical,Gram Stain,Cell Shape_x,Motility_x,Sporulation_x,Temperature Range_x,Optimal Temperature,Salinity_x,Oxygen Requirement,Habitat,Disease,Comments,Genome Quality Flags,biosample_title,contact_email,first_name,last_name,name,organism,publication_date,sample_accession,strain_x,submission_model,submission_package,submission_package_name,submitter_y,taxonomy_id,collection_date_x,culture_collection_x,geo_loc_name_x,isolation_source_x,sample_name_x,sample_type_x,type-material_x,comment_paragraph,depth_x,derived-from,env_broad_scale,env_local_scale,env_medium,environmental-sample,isolate_y,lat_lon_x,metagenome-source,metagenomic_x,sample-type,elev_x,environmetal-sample,collected_by_x,host_x,host_tissue_sampled_x,temp_x,env_biome_x,env_feature_x,env_material_x,isol_growth_condt_x,locus_tag_prefix_x,num_replicons_x,project_name_x,ref_biomaterial_x,16S recovered,16S recovery software,analysis project type,assembly quality,assembly software,bin parameters,binning software,completeness score,completeness software,contamination score,contig L50,decontamination software,note_x,number of contigs,reassembly post binning,relative coverage Anaerobic (SRR10097246),relative coverage Anoxic (SRR10375098),relative coverage Infiltration (SRR10097247),relative coverage Low pH (SRR10097245),relative coverage Oxic (SRR10375099),relative coverage Reference (SRR10375097),taxa id,total assembly size,nat_host,assembly_method_and_version,completeness_estimated_x,contamination_estimated_x,genome_coverage_x,mapping_method_and_version,quality_assessment_method_and_version,ENA-FIRST-PUBLIC_x,ENA-LAST-UPDATE_x,SUBJECT_ID_x,contact_lab,identified_by_x,environmental_sample_x,metagenome_source_x,biomaterial_provider_x,biome_x,feature_x,material_x,culture-collection_x,altitude_x,isolate_name_alias_x,host_disease_x,rel_to_oxygen_x,Alias_x,ENA checklist_x,INSDC center alias_x,INSDC center name_x,INSDC first public_x,INSDC last update_x,INSDC status_x,SRA accession_x,Sample Name_x,Title_x,serotype,subsrc_note_x,annotation_method,assembly_method_x,biosamplemodel,geographic location (country and/or atlantic ocean),geographic location (region and locality),mapping_method_x,quality_assessment_method_x,refinement_method,Cell Shape_y,Funding Program_x,GOLD Stamp ID_x,Gene Calling Method_x,Gram Staining_x,Isolation Comments_y,Isolation Site_x,Motility_y,Temperature Optimum_x,Temperature Range_y,Type Strain_y,alt_elev_x,biotic_relationship_x,country_x,environment_x,investigation_type_x,isolation-source_x,trophic_level_x,pH_x,Body Sample Site_y,Body Sample SubSite_x,Diseases_x,Host Name_y,Host Health_y,Sporulation_y,experimental_factor,lat-lon,ENA-CHECKLIST,External Id,Submitter Id,subsrc-note,collection date_x,environment (biome)_x,environment (feature)_x,environment (material)_x,geographic location (country and/or sea)_x,geographic location (latitude)_x,geographic location (longitude)_x,host common name,host-associated environmental package,investigation type_x,project name_x,sequencing method_x,STRAIN,description,complete

In [43]:
#merge.columns.duplicated()
merge_reordered = merge.reindex(sorted(merge.columns), axis=1)
merge_reordered.head(2)

,,# assembly_accession,16S recovered,16S recovery software,Alias_x,Alias_y,Altitude,ArrayExpress-Phenotype,ArrayExpress-SPECIES,ArrayExpress-STRAIN_OR_LINE,ArrayExpress-Species,ArrayExpress-StrainOrLine,Assembly Accession,Assembly Method_x,Assembly Method_y,BioProject Accession,BioSample Accession,BioSampleModel_x,BioSampleModel_y,Body Sample Site,Body Sample Site_x,Body Sample Site_y,Body Sample SubSite_x,Body Sample SubSite_y,Broker name,Cell Shape,Cell Shape_x,Cell Shape_y,Chromosomes,Collaborator ID,Collection Date,Comments,Completion Date,Contigs,Culture Collection,Depth_x,Description,Disease,Diseases_x,Diseases_y,ENA checklist_x,ENA checklist_y,ENA-CHECKLIST,ENA-FIRST-PUBLIC_x,ENA-FIRST-PUBLIC_y,ENA-LAST-UPDATE_x,ENA-LAST-UPDATE_y,External Id,Extraction Date,Extraction Method,FDA CVM ID_x,FDA CVM ID_y,Funding Program_x,Funding Program_y,GC Content,GOLD Stamp ID_x,GOLD Stamp ID_y,GenBank Accessions,Gene Calling Method_x,Gene Calling Method_y,Genome Coverage,Genome Length,Genome Name,Genome Quality Flags,Genome Status,Genome_ID,Genus,Geographic Location,Gram Stain,Gram Staining_x,Gram Staining_y,Habitat,Historical Monobactam Produced_x,Historical Monobactam Produced_y,Host Age,Host Gender,Host Health,Host Health_x,Host Health_y,Host Name,Host Name_x,Host Name_y,Host_x,Host_y,INSDC center alias_x,INSDC center alias_y,INSDC center name_x,INSDC center name_y,INSDC first public_x,INSDC first public_y,INSDC last update_x,INSDC last update_y,INSDC status_x,INSDC status_y,Isolation Comments,Isolation Comments_x,Isolation Comments_y,Isolation Country,Isolation Site_x,Isolation Site_y,Isolation Source,Isolation source_x,Isolation source_y,Laboratory Host,Latitude,Longitude,MAG_number,MLST,Motility,Motility_x,Motility_y,NCBI Taxon ID,Optimal Temperature,Organism Name,Other Clinical,Other Environmental,Other Typing,Oxygen Requirement,Passage Date,Pathovar,Phenotypes_x,Phenotypes_y,Plasmids,ProjectAccession,PublicAccession,Publication,RefSeq Accessions,RefSeq CDS,SRA Accession,SRA accession_x,SRA accession_y,STRAIN,SUBJECT_ID_x,SUBJECT_ID_y,Salinity (PSU),Salinity_x,Salinity_y,Sample Description,Sample Name_x,Sample Name_y,Sequencing Centers,Sequencing Depth,Sequencing Platform,Sequencing Status,Sequencing Technology,Serovar,Species,Sporulation,Sporulation_x,Sporulation_y,Strain,Strain_x,Submitter Id,Tax ID,Temperature Optimum_x,Temperature Optimum_y,Temperature Range,Temperature Range_x,Temperature Range_y,Title_x,Title_y,Type Strain,Type Strain_x,Type Strain_y,Type-material,Unique Monobactam Clusters Identified_x,Unique Monobactam Clusters Identified_y,WGA amplification approach,abs_air_humidity,air_temp,alkalinity (carbonate/bicarbonate),alt_elev_x,alt_elev_y,altitude_x,altitude_y,ammonium,analysis project type,annotation_method,anonymized_name_x,anonymized_name_y,asm_name,assembly quality,assembly software,assembly_accession,assembly_level,assembly_method_and_version,assembly_method_version_x,assembly_method_version_y,assembly_method_x,assembly_method_y,assembly_tool,assembly_x,assembly_y,attribute_package,bin parameters,bin_id,binning parameters,binning software,bio_material,biomass,biomaterial_provider,biomaterial_provider_x,biomaterial_provider_y,biome_x,biome_y,bioproject,biosample,biosample_title,biosamplemodel,biotic_relationship_x,biotic_relationship_y,body_site,build_occup_type,building_setting,carb_dioxide,carb_nitro_ratio,chlorophyll,collected-by,collected_by,collected_by_x,collected_by_y,collection date_x,collection date_y,collection-date,collection_date_x,collection_date_y,collection_room,comment_paragraph,common name,completeness,completeness score,completeness software,completeness_estimated_x,completeness_estimated_y,contact_email,contact_lab,contamination,contamination score,contamination_estimated_x,contamination_estimated_y,contig L50,country_x,country_y,culture collection,culture-collection_x,culture-collection_y,culture_collection_x,culture_collection_y,cyanobacterial_culture,decontamination software,depth_x,de

In [44]:
columns = merge_reordered.columns.to_list()

In [45]:
with open('Attributes.csv', 'w') as f:
    for item in columns:
        f.write("%s\n" % item)

**Open file that has columns to be joined together:**

In [46]:
with open("Attributes_reformated.csv", "r+") as f:
    lines = f.readlines()

In [47]:
#create dictionary with info from loaded file
d={}
for line in lines:
    words = line.split(",")
    words = [x.strip() for x in words if x.strip()]
    d[words[0]] = words[1:]

In [48]:
new_cols=[]
to_drop = []
dummy = merge_reordered.copy() # copy of the dataframe (for security reasons)

In [49]:
for i,a in d.items(): #i - name of new column; a - columns to merge; d : dictionary[i] = a
    #print("New column: " + i + "_merged; combining: " +  str(a))
    #print(i)
    to_drop.append(a[0])
    dummy[i + "_merged"] = dummy[a[0]]
    for x in a[1:]:
        dummy[i + "_merged"] = dummy[i + "_merged"].combine_first(dummy[x])
        #print("added " +  x)
        to_drop.append(x)
    new_cols.append(str(i) + "_merged")

In [50]:
to_look = dummy.reindex(sorted(dummy.columns), axis=1)
to_look.to_csv("before_cleaning.csv")

In [51]:
# drop columns already used in the joined columns:
dummy3 = dummy.drop(columns=to_drop)

# rename columns created by the join to the final name:
dummy3.columns = dummy3.columns.str.replace('_merged', '')
dummy3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2686 entries, 0 to 2685
Columns: 299 entries,  to WGS master
dtypes: float64(44), object(255)
memory usage: 6.1+ MB


In [52]:
length = len(dummy3.index)

In [53]:
dummy3.loc["SUM", :] = dummy3.isnull().sum(axis = 0)
#dummy3.to_csv("Dummy3.csv")

#remove columns that have less than 10 entries
dummy4 = dummy3[dummy3.columns[dummy3.loc["SUM"] <(length - 10)]]
dummy4.drop(index="SUM", inplace=True)
dummy4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2686 entries, 0 to 2685
Columns: 169 entries, Alias to WGS master
dtypes: float64(9), object(160)
memory usage: 3.5+ MB


/home/gomes/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
dummy4.head(2)

,Alias,Altitude,Anonymized name,Arrayexpress species,Arrayexpress strain or line,ASM name,Assembly accession,Assembly method,Assembly level,Biomaterial provider,Bioproject accession,Biosample accession,Biosample model,Biosample title,Biotic relationship,Body sample site,Cell shape,Chromosomes,Collected by,Collection date,Comment paragraph,Completeness estimated,Completion date,Contact email,Contact lab,Contamination estimated,Contigs,Country,Culture collection,Depth,Derived from,Description,Disease,Elevation,ENA checklist,ENA-first-public,ENA-last-update,Env biome,Env broad scale,Env feature,Env local scale,Env material,Env medium,Environment,Environmental sample,Estimated size,Excluded from refseq,FDA cvm ID,Finishing strategy (depth of coverage),First name,Ftp path,Gbrs paired ASM,Gc content,Genbank accessions,Gene calling method,Genome coverage,Genome ID,Genome length,Genome name,Genome quality flags,Genome rep,Genome status,Genotype,Geo loc name,Geographic location,Geographic location (country and/or atlantic ocean),GOLD stamp ID,Gram stain,Habitat,Host,Host age,Host disease,Host disease outcome,Host gender,Host health,Host name,Host scientific name,Host sex,Host taxid,Host tissue sampled,Identification method,Identified by,Infraspecific name,INSDC center alias,INSDC center name,INSDC first public,INSDC last update,INSDC status,Investigation type,Isol growth condt,Isolate,Isolation comments,Isolation country,Isolation site,Isolation source,Last name,lat lon,Latitude,Locus tag prefix,Longitude,Mapping method and version,Material,Metagenome source,Metagenomic,Misc param,Misc param: hmp supersite,Mlst,Motility,Name,Ncbi taxon ID,Note,Num replicons,Organism,Organism name,Other clinical,Other environmental,Oxygen requirement,Paired ASM comp,pH,Phenotypes,Plasmids,Project name,Project type,Publication,Publication date,Quality assessment method and version,Ref biomaterial,Refseq accessions,Refseq category,Refseq cds,Rel to oxygen,Relation to type material,Release type,Salinity,Samp size,Sample accession,Sample name,Sample type,Seq rel date,Sequencing centers,Sequencing depth,Sequencing method,Sequencing platform,Sequencing status,Sop,Source mat ID,Species taxid,Specific host,Sporulation,SRA accession,Strain,Strain name alias,Subject ID,Submission model,Submission package,Submitter,Subsrc note,Supplier name,Taxid,Temperature,Temperature optimum,Temperature range,Title,Trophic level,Type strain,Type material,Value,Version status,WGS master
0,NaN,NaN,NaN,NaN,NaN,ASM1664v1,GCA_000016645.1,NaN,Complete Genome,NaN,PRJNA16082,SAMN02598357,NaN,Sample from Flavobacterium johnsoniae UW101,NaN,NaN,Rod,1.0,NaN,NaN,Flavobacterium johnsoniae ATCC 17061. Flavobac...,NaN,2007-05-01T00:00:00Z,NaN,NaN,NaN,1.0,NaN,ATCC 17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,GCF_000016645.1,34.1,CP000685,NaN,NaN,GCA_000016645.1,6096872.0,Flavobacterium johnsoniae UW101,NaN,Full,Complete,NaN,NaN,United Kingdom: England,NaN,NaN,NaN,Multiple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,strain=UW101; ATCC 17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,isolated from soil in England and is the type ...,United Kingdom,NaN,soil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NCBI,376686.0,NaN,NaN,Flavobacterium johnsoniae UW101,Flavobacterium johnsoniae UW101,NaN,NaN,Aerobic,identical,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-28T00:00:00.000,NaN,NaN,NC_009441,representative genome,5017.0,NaN,assembly from type material,Major,Non-halophilic,NaN,SAMN02598357,CP000685,NaN,2007/05/01,DOE Joint Genome Institute,NaN,NaN,NaN,complete,NaN,NaN,986.0,NaN,No,NaN,ATCC 17061,NaN,NaN,Generic,Generic.1.0,US DOE Joint Genome Institute,NaN,NaN,376686,NaN,20-30,Mesophilic,NaN,NaN,Yes,type strain of Flavobacterium johnsoniae,NaN,latest,NaN
1,NaN,NaN,NaN,NaN,NaN,ASM2328v1,GCA_000023285.1,NaN,Complete Genome,NaN,PRJNA29403,SAMN00001911,NaN,Generic sample from Capnocytophaga ochracea DS...,NaN,NaN,Rod,1.

In [55]:
dummy4.drop(columns=["Completeness estimated","Contamination estimated","Gc content","Genome length","Genome coverage"], inplace=True)

/home/gomes/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [56]:
dummy4.to_csv("Metadata.csv", index=False)

## 2.6 Check if sample is marine or not 
From coordinates (latitude and longitude). \
Python package: global_land_mask

In [57]:
metadata = dummy4.copy()

In [58]:
pattern = '|'.join([" N"," W", " S", " E", " DD", "N", "E"])
metadata["latitude_longitude"] = metadata["lat lon"].str.replace(pattern, "")

In [59]:
metadata["Lat"] = metadata["Latitude"].str.replace(" DD", "")
metadata["Long"] = metadata["Longitude"].str.replace(" DD", "")

In [60]:
new = metadata["latitude_longitude"].str.split(" ", expand=True)

In [61]:
# making separate column names
metadata["lat"]= new[0] 
metadata["lon"]= new[1] 

In [62]:
metadata["Latitude"] = metadata["Lat"].combine_first(metadata["lat"])
metadata["Longitude"] = metadata["Long"].combine_first(metadata["lon"])
metadata.drop(columns=["latitude_longitude","Lat", "Long", "lat lon", "lat", "lon"], inplace=True)

In [63]:
df = metadata[["Assembly accession", "Latitude", "Longitude"]]

In [64]:
from global_land_mask import globe
d = {}
for i in df.iterrows():
    for x in i[1:]:
        accession = x[0]
        #print(accession)
        lat = (x[1])
        #print(lat)
        lon = (x[2])
        #print(lon)
        try:
            lat = float(lat)
            lon = float(lon)
            try:
                is_on_land = globe.is_land(lat,lon)
                d[accession] = is_on_land
                #print("added to dic")
            except:
                d[accession] = None
                #print("out of bonds")
        except ValueError:
            d[accession] = None
            #print("Not a float")
        #print("\n")

In [65]:
is_marine = pd.DataFrame.from_dict(d, orient='index', columns=["Marine_coordinates"])

In [66]:
merge = pd.merge(metadata, is_marine, how="left", left_on="Assembly accession", right_index=True, indicator="merge")
merge["merge"].value_counts()

both          2686
right_only       0
left_only        0
Name: merge, dtype: int64

In [67]:
merge["Marine_coordinates"].value_counts()

True     393
False    270
Name: Marine_coordinates, dtype: int64

In [68]:
merge = merge.drop(columns=["merge"])
merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2686 entries, 0 to 2685
Columns: 164 entries, Alias to Marine_coordinates
dtypes: float64(7), object(157)
memory usage: 3.4+ MB


In [69]:
merge = merge.set_index("Assembly accession")
merge.to_csv("Metadata.csv")

## 2.7 Classifier

after running classifier_v3.0.py:

In [86]:
metadata_class = pd.read_csv("Metadata_classification.csv")
metadata_class.head(2)

,Assembly accession,Alias,Altitude,Anonymized name,Arrayexpress species,Arrayexpress strain or line,ASM name,Assembly method,Assembly level,Biomaterial provider,Bioproject accession,Biosample accession,Biosample model,Biosample title,Biotic relationship,Body sample site,Cell shape,Chromosomes,Collected by,Collection date,Comment paragraph,Completion date,Contact email,Contact lab,Contigs,Country,Culture collection,Depth,Derived from,Description,Disease,Elevation,ENA checklist,ENA-first-public,ENA-last-update,Env biome,Env broad scale,Env feature,Env local scale,Env material,Env medium,Environment,Environmental sample,Estimated size,Excluded from refseq,FDA cvm ID,Finishing strategy (depth of coverage),First name,Ftp path,Gbrs paired ASM,Genbank accessions,Gene calling method,Genome ID,Genome name,Genome quality flags,Genome rep,Genome status,Genotype,Geo loc name,Geographic location,Geographic location (country and/or atlantic ocean),GOLD stamp ID,Gram stain,Habitat,Host,Host age,Host disease,Host disease outcome,Host gender,Host health,Host name,Host scientific name,Host sex,Host taxid,Host tissue sampled,Identification method,Identified by,Infraspecific name,INSDC center alias,INSDC center name,INSDC first public,INSDC last update,INSDC status,Investigation type,Isol growth condt,Isolate,Isolation comments,Isolation country,Isolation site,Isolation source,Last name,Latitude,Locus tag prefix,Longitude,Mapping method and version,Material,Metagenome source,Metagenomic,Misc param,Misc param: hmp supersite,Mlst,Motility,Name,Ncbi taxon ID,Note,Num replicons,Organism,Organism name,Other clinical,Other environmental,Oxygen requirement,Paired ASM comp,pH,Phenotypes,Plasmids,Project name,Project type,Publication,Publication date,Quality assessment method and version,Ref biomaterial,Refseq accessions,Refseq category,Refseq cds,Rel to oxygen,Relation to type material,Release type,Salinity,Samp size,Sample accession,Sample name,Sample type,Seq rel date,Sequencing centers,Sequencing depth,Sequencing method,Sequencing platform,Sequencing status,Serovar,Sop,Source mat ID,Species taxid,Specific host,Sporulation,SRA accession,Strain,Strain name alias,Subject ID,Submission model,Submission package,Submitter,Subsrc note,Supplier name,Taxid,Temperature,Temperature optimum,Temperature range,Title,Trophic level,Type strain,Type material,Value,Version status,WGS master,Marine_coordinates,MAG_words,MAG,Human-associated_words,Human-associated,Terrestrial_words,Terrestrial,Marine_words,Marine,Freshwater_words,Freshwater,Origin,Conflict_words,Conflict?
0,GCA_000016645.1,NaN,NaN,NaN,NaN,NaN,ASM1664v1,NaN,Complete Genome,NaN,PRJNA16082,SAMN02598357,NaN,Sample from Flavobacterium johnsoniae UW101,NaN,NaN,Rod,1.0,NaN,NaN,Flavobacterium johnsoniae ATCC 17061. Flavobac...,2007-05-01T00:00:00Z,NaN,NaN,1.0,NaN,ATCC 17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,GCF_000016645.1,CP000685,NaN,GCA_000016645.1,Flavobacterium johnsoniae UW101,NaN,Full,Complete,NaN,NaN,United Kingdom: England,NaN,NaN,NaN,Multiple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,strain=UW101; ATCC 17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,isolated from soil in England and is the type ...,United Kingdom,NaN,soil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NCBI,376686.0,NaN,NaN,Flavobacterium johnsoniae UW101,Flavobacterium johnsoniae UW101,NaN,NaN,Aerobic,identical,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-28T00:00:00.000,NaN,NaN,NC_009441,representative genome,5017.0,NaN,assembly from type material,Major,Non-halophilic,NaN,SAMN02598357,CP000685,NaN,2007/05/01,DOE Joint Genome Institute,NaN,NaN,NaN,complete,NaN,NaN,NaN,986.0,NaN,No,NaN,ATCC 17061,NaN,NaN,Generic,Generic.1.0,US DOE Joint Genome Institute,NaN,NaN,376686.0,NaN,20-30,Mesophilic,NaN,NaN,Yes,type strain of Flavobacterium johnsoniae,NaN,latest,NaN,NaN,NaN,NaN,NaN,NaN,['soil'],Terrestrial,NaN,NaN,NaN,NaN,Terrestrial,Terrestrial,NaN
1,GCA_000

In [87]:
metadata_class["Conflict?"].value_counts()

Yes    7
Name: Conflict?, dtype: int64

In [88]:
metadata_class["Conflict?"] = ["Unclassified" if str(x) == "Yes" else None for x in metadata_class['Conflict?']]
metadata_class["Conflict?"].value_counts()

Unclassified    7
Name: Conflict?, dtype: int64

In [89]:
metadata_class["Origin"] = metadata_class["Conflict?"].combine_first(metadata_class["Origin"])
metadata_class["Origin"] = ["Non_marine" if  str(x) == "Terrestrial" or str(x) =="Freshwater" or str(x) =="Human-associated" else "Marine" if str(x)=="Marine" else "Unclassified" for x in metadata_class['Origin']]
metadata_class["Origin"].value_counts()

Unclassified    1594
Marine           660
Non_marine       432
Name: Origin, dtype: int64

In [90]:
metadata_class.drop(columns=["MAG_words","Human-associated_words","Human-associated","Terrestrial_words","Terrestrial","Marine_words","Marine","Freshwater_words","Freshwater","Conflict_words","Conflict?"], inplace=True)

In [91]:
metadata_class.head()

,Assembly accession,Alias,Altitude,Anonymized name,Arrayexpress species,Arrayexpress strain or line,ASM name,Assembly method,Assembly level,Biomaterial provider,Bioproject accession,Biosample accession,Biosample model,Biosample title,Biotic relationship,Body sample site,Cell shape,Chromosomes,Collected by,Collection date,Comment paragraph,Completion date,Contact email,Contact lab,Contigs,Country,Culture collection,Depth,Derived from,Description,Disease,Elevation,ENA checklist,ENA-first-public,ENA-last-update,Env biome,Env broad scale,Env feature,Env local scale,Env material,Env medium,Environment,Environmental sample,Estimated size,Excluded from refseq,FDA cvm ID,Finishing strategy (depth of coverage),First name,Ftp path,Gbrs paired ASM,Genbank accessions,Gene calling method,Genome ID,Genome name,Genome quality flags,Genome rep,Genome status,Genotype,Geo loc name,Geographic location,Geographic location (country and/or atlantic ocean),GOLD stamp ID,Gram stain,Habitat,Host,Host age,Host disease,Host disease outcome,Host gender,Host health,Host name,Host scientific name,Host sex,Host taxid,Host tissue sampled,Identification method,Identified by,Infraspecific name,INSDC center alias,INSDC center name,INSDC first public,INSDC last update,INSDC status,Investigation type,Isol growth condt,Isolate,Isolation comments,Isolation country,Isolation site,Isolation source,Last name,Latitude,Locus tag prefix,Longitude,Mapping method and version,Material,Metagenome source,Metagenomic,Misc param,Misc param: hmp supersite,Mlst,Motility,Name,Ncbi taxon ID,Note,Num replicons,Organism,Organism name,Other clinical,Other environmental,Oxygen requirement,Paired ASM comp,pH,Phenotypes,Plasmids,Project name,Project type,Publication,Publication date,Quality assessment method and version,Ref biomaterial,Refseq accessions,Refseq category,Refseq cds,Rel to oxygen,Relation to type material,Release type,Salinity,Samp size,Sample accession,Sample name,Sample type,Seq rel date,Sequencing centers,Sequencing depth,Sequencing method,Sequencing platform,Sequencing status,Serovar,Sop,Source mat ID,Species taxid,Specific host,Sporulation,SRA accession,Strain,Strain name alias,Subject ID,Submission model,Submission package,Submitter,Subsrc note,Supplier name,Taxid,Temperature,Temperature optimum,Temperature range,Title,Trophic level,Type strain,Type material,Value,Version status,WGS master,Marine_coordinates,MAG,Origin
0,GCA_000016645.1,NaN,NaN,NaN,NaN,NaN,ASM1664v1,NaN,Complete Genome,NaN,PRJNA16082,SAMN02598357,NaN,Sample from Flavobacterium johnsoniae UW101,NaN,NaN,Rod,1.0,NaN,NaN,Flavobacterium johnsoniae ATCC 17061. Flavobac...,2007-05-01T00:00:00Z,NaN,NaN,1.0,NaN,ATCC 17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,GCF_000016645.1,CP000685,NaN,GCA_000016645.1,Flavobacterium johnsoniae UW101,NaN,Full,Complete,NaN,NaN,United Kingdom: England,NaN,NaN,NaN,Multiple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,strain=UW101; ATCC 17061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,isolated from soil in England and is the type ...,United Kingdom,NaN,soil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NCBI,376686.0,NaN,NaN,Flavobacterium johnsoniae UW101,Flavobacterium johnsoniae UW101,NaN,NaN,Aerobic,identical,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-28T00:00:00.000,NaN,NaN,NC_009441,representative genome,5017.0,NaN,assembly from type material,Major,Non-halophilic,NaN,SAMN02598357,CP000685,NaN,2007/05/01,DOE Joint Genome Institute,NaN,NaN,NaN,complete,NaN,NaN,NaN,986.0,NaN,No,NaN,ATCC 17061,NaN,NaN,Generic,Generic.1.0,US DOE Joint Genome Institute,NaN,NaN,376686.0,NaN,20-30,Mesophilic,NaN,NaN,Yes,type strain of Flavobacterium johnsoniae,NaN,latest,NaN,NaN,NaN,Non_marine
1,GCA_000023285.1,NaN,NaN,NaN,NaN,NaN,ASM2328v1,NaN,Complete Genome,NaN,PRJNA29403,SAMN00001911,NaN,Generic sample from Capnocytophaga ochracea DS...,NaN,NaN,Rod,1.0,NaN,NaN,Capnocytophaga ochracea DSM 7271,2009-04-30T00:00:00Z,

In [92]:
metadata_class.to_csv("Metadata_final.csv", index=False)